# 数据分析

In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np

In [2]:
train_file = "/data/sulixin/research/contest/MSMARCO/data/datasets/msmarcov2/train_v2.1.json"
dev_file = "/data/sulixin/research/contest/MSMARCO/data/datasets/msmarcov2/dev_v2.1.json"
test_file = "/data/sulixin/research/contest/MSMARCO/data/datasets/msmarcov2/eval_v2.1_public.json"
#数据读入
def load_dataset(filename):
    data = json.load(open(filename))
    print( '%s has %s samples' % (os.path.basename(filename),len(data['query'])  ))
    return data 

In [3]:
train = load_dataset(train_file)
dev = load_dataset(dev_file)
test = load_dataset(test_file)

train_v2.1.json has 808731 samples
dev_v2.1.json has 101093 samples
eval_v2.1_public.json has 101092 samples


In [6]:
train.keys()

dict_keys(['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'])

In [9]:
list(train['passages'].values())[1]

[{'is_selected': 0,
  'passage_text': 'group discussions, community boards or panels with a third party, or victim and offender dialogues, and requires a skilled facilitator who also has sufficient understanding of sexual assault, domestic violence, and dating violence, as well as trauma and safety issues.',
  'url': 'https://www.justice.gov/ovw/file/926101/download'},
 {'is_selected': 0,
  'passage_text': "punishment designed to repair the damage done to the victim and community by an offender's criminal act. Ex: community service, Big Brother program indeterminate sentence",
  'url': 'https://quizlet.com/1128245/criminal-justice-exam-1-flash-cards/'},
 {'is_selected': 0,
  'passage_text': 'Tutorial: Introduction to Restorative Justice. Restorative justice is a theory of justice that emphasizes repairing the harm caused by criminal behaviour. It is best accomplished through cooperative processes that include all stakeholders. This can lead to transformation of people, relationships an

In [12]:
def count_q(data):
    print (sum( [len(d) for d in data.values()]))
count_q( train['passages'])

8069749


In [39]:
print(train_df.iloc[100]['query'])
train_df.iloc[100]['passages']

quizlet how did existing industrial workers view immigrant industrial workers?


[{'is_selected': 0,
  'passage_text': 'At this time the Industrial Workers of the World had a membership of over 100,000 members. In 1913 William Haywood replaced Vincent Saint John as secretary-treasurer of the Industrial Workers of the World. By this time, the IWW had 100,000 members.',
  'url': 'http://spartacus-educational.com/USAiww.htm'},
 {'is_selected': 0,
  'passage_text': "This was not true of the Industrial Workers of the World and as a result many of its members were first and second generation immigrants. Several immigrants such as Mary 'Mother' Jones, Hubert Harrison, Carlo Tresca, Arturo Giovannitti and Joe Haaglund Hill became leaders of the organization.",
  'url': 'http://spartacus-educational.com/USAiww.htm'},
 {'is_selected': 0,
  'passage_text': 'Chinese Immigration and the Chinese Exclusion Acts. In the 1850s, Chinese workers migrated to the United States, first to work in the gold mines, but also to take agricultural jobs, and factory work, especially in the garm

In [42]:
def text_len(text):
    return len(text.split())

def list_len(li):
    return len(li)

def avg_text_len(passages):
    lens = [len(p['passage_text'].split()) for p in passages]
    return sum(lens) * 1.0 / len(lens)

def avg_len(texts):
    lens = [len(t.split()) for t in texts]
    return sum(lens) * 1.0 / len(lens)

def select_cnt(ps):
    return sum([p['is_selected'] for p in ps])

def x_hit(idx):
    def func(ps):
        if idx >= len(ps):
            return None
        return ps[idx]['is_selected']
    return func

def has_answer(passages):
    return len([p for p in passages if p['is_selected']]) > 0 


def my_stat(data):
    train_df['q_len'] = train_df['query'].apply(text_len)
    train_df['a_cnt'] = train_df['answers'].apply(list_len)
    train_df['a_len'] = train_df['answers'].apply(avg_len)
    train_df['p_cnt'] = train_df['passages'].apply(list_len)
    train_df['p_len'] = train_df['passages'].apply(avg_text_len)
    train_df['select'] = train_df['passages'].apply(select_cnt)
    train_df['has_answer'] = train_df['passages'].apply(has_answer)
    for i in range(10):
        train_df['%s_hit' % i] = train_df['passages'].apply(x_hit(i))
    print('-----overall----')
    print(train_df.describe())
    print('-----answerable------')
    print(train_df[train_df['has_answer']==1].describe())
    print('-----unanswreable-----')
    print(train_df[train_df['has_answer']==0].describe())

In [43]:
my_stat(train)

-----overall----
           query_id          q_len          a_cnt          p_cnt  \
count  8.087310e+05  808731.000000  808731.000000  808731.000000   
mean   5.704129e+05       6.365910       1.017121       9.978286   
std    3.350953e+05       2.648773       0.130339       0.375860   
min    1.000000e+00       1.000000       1.000000       1.000000   
25%    2.825540e+05       5.000000       1.000000      10.000000   
50%    5.615370e+05       6.000000       1.000000      10.000000   
75%    8.576385e+05       8.000000       1.000000      10.000000   
max    1.185869e+06      75.000000       6.000000      27.000000   

               p_len          a_len         select          0_hit  \
count  808731.000000  808731.000000  808731.000000  808731.000000   
mean       56.478839       9.151124       0.658762       0.121996   
std        17.640566      11.294580       0.559099       0.327281   
min        11.000000       0.000000       0.000000       0.000000   
25%        45.600000     